# 1. Data loading

In [1]:
from src.data import get_questions_df

question_df = get_questions_df()

/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/src/data.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Category"] = datasets[sheet_name]["Category"+suffix]
/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/src/data.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Category"] = datasets[sheet_name]["Category"+suffix]
/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/src/data.py:28: SettingWithCopyWarning: 
A value is trying to be set on a cop

# 2. LLM challenger Answer a question

In [2]:
from src.consts import SELECTED_LLMS
from src.answer import llms_answer_questions
from src.orchestrator import (
    get_missing_question_ids, 
    load_answered_questions, BATCH_SIZE,
    save_answers)

/home/lambert-bruyas/miniconda3/envs/gpt_pursuit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
answered_questions = load_answered_questions()

In [4]:
missing_ids = get_missing_question_ids(
    answered_questions,
    question_df["id"].tolist()
)

In [5]:
question_sample = question_df.set_index('id').loc[missing_ids].sample(BATCH_SIZE)

In [6]:
question_answers = llms_answer_questions(question_sample["Question"].tolist(), SELECTED_LLMS)

/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/HuggingFaceTB/SmolLM2-360M-Instruct


Device set to use cpu


/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/HuggingFaceTB/SmolLM2-135M-Instruct


Device set to use cpu


/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/google/flan-t5-small


...

Device set to use cpu


/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/google/gemma-2-2b-it


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 11.43it/s]
Device set to use cpu


/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/Qwen/Qwen2.5-1.5B-Instruct


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.54it/s]
Device set to use cpu


In [8]:
for i, llm in enumerate(SELECTED_LLMS):
    model_answers:list[str] = []
    for j, question in enumerate(question_answers[i]):
        answer = question_answers[i][j][0]['generated_text']
        if isinstance(answer, list):
            answer = answer[-1]['content']
        else:
            pass
        model_answers.append(answer)
        
    question_sample.loc[:, llm["pipeline_kwargs"]["model"]] = model_answers

In [17]:
import difflib
from typing import TypedDict, NotRequired
import pandas as pd
from src.types import Question
from datetime import datetime

question_output:list[Question] = []

for i in range(len(question_sample)):
    question_output.append({
        "id": int(question_sample.iloc[i].name),
        "category": question_sample.iloc[i]["Category"],
        "question": question_sample.iloc[i]["Question"],
        "expected_answer": question_sample.iloc[i]["Answer"],
        "llm_answers": {llm["pipeline_kwargs"]["model"]: {
            "answer": question_sample.iloc[i].loc[llm["pipeline_kwargs"]["model"]], 
            "is_correct": difflib.SequenceMatcher(None, question_sample.iloc[i]["Answer"], question_sample.iloc[i].loc[llm["pipeline_kwargs"]["model"]]).ratio() > 0.8} for llm in SELECTED_LLMS},
        "difficulty": question_sample.iloc[i]["Difficulty"] if not pd.isna(question_sample.iloc[i]["Difficulty"]) else None
    })
save_answers(question_output)


FileNotFoundError: [Errno 2] No such file or directory: '/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/data/answers/2025-12-14 19:24:56.031952.json'

In [21]:
import json
import os 
ANSWER_FOLDER = "/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/data/answers"


def save_answers(questions:list[Question])->None:
    json.dump(
        questions,
        open(os.path.join(ANSWER_FOLDER, f"{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"), 
        "w"),
        indent=4)
    
save_answers(question_output)